In [ ]:
%pip install langchain
%pip install sentence_transformers
%pip install langchain-chroma
%pip install langchain_community

In [ ]:
import os
os.environ["HF_TOKEN"] = "Your_Hugging_Face_token"

In [ ]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
model_name = "Alibaba-NLP/gte-large-en-v1.5"
model_kwargs = {'device': 'cpu', 'trust_remote_code':'True'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
model = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

In [98]:
from langchain.vectorstores import Chroma
vectordb = Chroma(persist_directory="embeddings_directory", embedding_function=model)

In [99]:
# LangChain supports many other chat models. Here, we're using Ollama
from langchain_community.chat_models import ChatOllama

# supports many more optional parameters. Hover on your `ChatOllama(...)`
# class to view the latest available supported parameters
llm = ChatOllama(model="qwen2")

In [100]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [101]:
TEMPLATE = """Answer the question based ONLY on the following context:
{context}
Question: {question}
Do not mention "based on the context" or similar in the output you generate. 
"""
prompt = ChatPromptTemplate.from_template(TEMPLATE)

In [102]:
retriever = MultiQueryRetriever.from_llm(
    vectordb.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

In [103]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [104]:
chain.invoke("who wrote constitution?")

'The Constitution of India was written by a Constituent Assembly that included representatives from various political parties and social groups across India, under the chairmanship of Dr. Bhimrao Ambedkar. The drafting process took nearly three years, starting from August 1947 until November 1949.'